# web scrape Anne Will Gäste

## Install packages

In [14]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install --user bs4 
!{sys.executable} -m pip install --user requests nltk nameparser

## import packages and get archive website

In [3]:
import re
from bs4 import BeautifulSoup
import requests
import json
import re
url = 'https://daserste.ndr.de/annewill/archiv/index.html' #Sendungsarchiv von Anne Will
response = requests.get(url, timeout=5)
content = BeautifulSoup(response.content, "html.parser")

### find all paragraphes that contain the class "teasertext"

In [4]:
content.find_all("p", class_= "teasertext")

[<p class="teasertext">
 <a href="/annewill/archiv/Streiken-statt-Pauken-aendert-die-Generation-Greta-die-Politik,erste11444.html" title="Zum Artikel: Streiken statt Pauken - ändert die Generation Greta die Politik?">
 Anne Will im Gespräch mit Greta Thunberg. Gäste im Studio: Robert Habeck, Reiner Haseloff, Wolfgang Kubicki, Therese Kah und Harald Lesch.
 </a> | <strong><a href="/annewill/archiv/Streiken-statt-Pauken-aendert-die-Generation-Greta-die-Politik,erste11444.html" title="Zum Artikel: Streiken statt Pauken - ändert die Generation Greta die Politik?">mehr</a></strong>
 </p>, <p class="teasertext">
 <a href="/annewill/archiv/Zwischen-Hoechstleistung-und-Ueberlastung-wann-macht-Arbeit-krank,erste11442.html" title="Zum Artikel: Zwischen Höchstleistung und Überlastung – wann macht Arbeit krank?">
 Mit Sahra Wagenknecht, Thomas de Maizière, Katja Suding, Alexander Jorde und Klaus Lieb.
 </a> | <strong><a href="/annewill/archiv/Zwischen-Hoechstleistung-und-Ueberlastung-wann-macht-Ar

### extract the link text out of the paragraphs

In [5]:
list1 =content.find_all("p", class_="teasertext")
list1[0].find("a").contents

['\nAnne Will im Gespräch mit Greta Thunberg. Gäste im Studio: Robert Habeck, Reiner Haseloff, Wolfgang Kubicki, Therese Kah und Harald Lesch.\n']

### text clean als string

In [15]:
text = list1[0].find("a").contents
str(text).split("\\n")[1]

'Anne Will im Gespräch mit Greta Thunberg. Gäste im Studio: Robert Habeck, Reiner Haseloff, Wolfgang Kubicki, Therese Kah und Harald Lesch.'

## Hier ganz billiger Filter mit replace() und strip()

In [17]:
for element in list1:
    text =element.find("a").contents
    cleanTxt= str(text).split("\\n")[1].replace("Mit", "").replace("mit", "").replace("im", "").replace("Anne Will", "").rstrip()
    cleanTxt = cleanTxt.replace("Gespräch", "").replace("Gäste", "").replace("Studio", "").strip().replace(".", "").replace(":", "").replace("und", ",").replace("  ", ",").replace(",,",",").strip()
    cleanTxt.replace(", ",",").replace(" ,",",").split(",")
    print(cleanTxt.replace(", ",",").replace(" ,",",").split(","))

['Greta Thunberg', 'Robert Habeck', 'Reiner Haseloff', 'Wolfgang Kubicki', 'Therese Kah', 'Harald Lesch']
['Sahra Wagenknecht', 'Thomas de Maizière', 'Katja Suding', 'Alexander Jorde', 'Klaus Lieb']
['Manfred Weber', 'Yanis Varoufakis', 'Beatrix von Storch', 'Cathrin Kahlweit', 'Christian Lindner']
['Malu Dreyer', 'Mike Mohring', 'Katrin Göring-Eckardt', 'Petra Vogel', 'Reinhold von Eben-Worlée', 'als Publikumsgast Guido Fahrendholz']
['Bischof Stephan Ackermann', 'Agnes Wich', 'Johannes-Wilhelm Erik', 'Matthias Katsch', 'Heribert Prantl']
['Heiko Maas', 'Sev Dağdelen', 'Jürgen Trittin', 'Georg Mascolo', 'Constanze Stelzenmüller']


#### ganz ok, bis auf sowas wie "als Publikumsgast"

# Richtig heavy nameparser mit tokenizer und shizzle

Quelle: 
google python extract name from text

https://stackoverflow.com/questions/20290870/improving-the-extraction-of-human-names-with-nltk

## weiterführender Artikel zu nltk und spacy
https://towardsdatascience.com/named-entity-recognition-with-nltk-and-spacy-8c4a7d88e7da


In [8]:
import nltk
from nameparser.parser import HumanName

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

def get_human_names(text):
    tokens = nltk.tokenize.word_tokenize(text)
    pos = nltk.pos_tag(tokens)
    sentt = nltk.ne_chunk(pos, binary = False)
    person_list = []
    person = []
    name = ""
    for subtree in sentt.subtrees(filter=lambda t: t.label() == 'PERSON'):
        for leaf in subtree.leaves():
            person.append(leaf[0])
        if len(person) > 1: #avoid grabbing lone surnames
            for part in person:
                name += part + ' '
            if name[:-1] not in person_list:
                person_list.append(name[:-1])
            name = ''
        person = []

    return (person_list)

[nltk_data] Downloading package punkt to /home/silvio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/silvio/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/silvio/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/silvio/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [9]:
for element in list1:
    text = element.find("a").contents
    print(get_human_names(str(text).split("\\n")[1]))

['Greta Thunberg', 'Robert Habeck', 'Reiner Haseloff', 'Wolfgang Kubicki', 'Harald Lesch']
['Sahra Wagenknecht', 'Katja Suding', 'Alexander Jorde', 'Klaus Lieb']
['Manfred Weber', 'Yanis Varoufakis', 'Cathrin Kahlweit', 'Christian Lindner']
['Malu Dreyer', 'Mike Mohring', 'Katrin Göring-Eckardt', 'Petra Vogel']
['Bischof Stephan Ackermann', 'Agnes Wich', 'Matthias Katsch', 'Heribert Prantl']
['Heiko Maas', 'Sevim Dağdelen', 'Jürgen Trittin', 'Georg Mascolo']


#### joar ist doch echt 1 nicer output